In [1]:
import numpy as np
from scipy.sparse.linalg import eigsh
from itertools import product
from Main_CC_files import functions as func
import pandas as pd
from functools import reduce
import os
from Main_CC_files import hamiltonianGenerator as hg
from Main_CC_files import ham_no_csv as HB
# import hamiltonian_Bryan as HB



# Show full DataFrame (no row/column truncation)
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', 0)            # Auto-detect line width
pd.set_option('display.max_colwidth', None)  # Full column width (if columns contain strings)

ModuleNotFoundError: No module named 'functions'

# Applying the delta function operation.

## Applying the delta function operation.(Work in Progress)

In [2]:
def generate_matrix(site, state):
    states = ["".join(map(str, digits)) for digits in product(range(state), repeat=site)]
    matrix = []

    print(states)

    for row in states:
        row_entries = [f"{row}|{col}" for col in states]
        matrix.append(row_entries)

    return matrix


def print_matrix(matrix):
    for row in matrix:
        print(" ".join(row))

In [3]:
def encode_tuple(digits):
    """Efficiently convert a tuple of digits to a single integer (e.g., (0,1,0) → 10)."""
    result = 0
    for d in digits:
        result = result * 10 + d
    return result

def generate_matrix(site, state):
    # Generate all basis states as tuples of digits
    states = list(product(range(state), repeat=site))
    

    # Precompute integer encoding for each state
    encoded = [encode_tuple(s) for s in states]

    # Compute digit shift for concatenation
    digit_scale = 10 ** site

    # Build matrix using pure integer math
    matrix = np.array([[row * digit_scale + col for col in encoded] for row in encoded])
    return matrix

In [4]:
matrix_3 = generate_matrix(site, state)
print(matrix_3)

NameError: name 'site' is not defined

In [ ]:
import numpy as np

# Sample tensor of shape (2, 3, 8, 8)
T = np.random.rand(2, 3, 8, 8)

# Use broadcasting to compare across n
comparison = T[0] != T[1]  # shape: (3, 8, 8), boolean

# Convert True/False to 1/0
comparison_int = comparison.astype(int)  # shape: (3, 8, 8)

In [ ]:
T = np.arange(3**3).reshape(3, 3, 3)

for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 4):
            T[i-1,j-1,k-1] = 100*i+10*j+k



# Contract i with k: enforce i == k
# T_masked = np.einsum('ij,kl,ijkl->ijkl', np.eye(3), np.eye(3), T)

In [ ]:
T = np.arange(3**2).reshape(3, 3)
print("Original Tensor (flattened):")
print(T.flatten())

for i in range(1, 4):
    for j in range(1, 4):
        T[i-1,j-1] = 10*i+1*j

matrices = [T, np.eye(3), np.eye(3)]
T_new = reduce(np.kron, matrices)

print(matrices)
print(T)
print(T_new.shape)

Original Tensor (flattened):
[0 1 2 3 4 5 6 7 8]
[array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])]
[[11 12 13]
 [21 22 23]
 [31 32 33]]
(27, 27)


In [ ]:
matrices = [T, np.eye(3), np.eye(3)]
matrices[0] = T
print(matrices)

print(np.eye(3).shape[0])

[array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])]
3


## Applying the delta function operation.(Finished Kinetic)

In [ ]:
def compute_matrix_individual_sites(K: np.ndarray, target_site: int, num_site: int, num_state: int)-> np.ndarray:

    matrices = [np.eye(2*num_state + 1)]*num_site
    matrices[target_site] = K

    K = reduce(np.kron, matrices)

    return K

def compute_kinetic_matrix(K: np.ndarray, num_site: int, num_state: int)-> np.ndarray:

    K_final = np.zeros(((2*num_state + 1)**num_site, (2*num_state + 1)**num_site))

    for target_site in range(num_site):

        K_final += compute_matrix_individual_sites(K, target_site, num_site, num_state)

    return K_final

In [ ]:
# site = 3
# state = 1
# K_baka = np.diag(np.arange(1,4))

# print(compute_kinetic_matrix(K_baka, site, state).shape)

## Applying the delta function operation.(Finished Potential)

In [ ]:
def kron_all(ops):
    return reduce(np.kron, ops)

def compute_nearest_neighbor_potential(V: np.ndarray, num_site: int, num_state: int) -> np.ndarray:
    """
    V: 4D tensor (num_state, num_state, num_state, num_state)
       representing the 2-site potential: V[i, j, k, l] = ⟨ij|V|kl⟩
    """
    num_state = 2*num_state +1

    dim = num_state ** num_site
    V_Final = np.zeros((dim, dim))
    print(num_state)

    V_matrix = V.reshape(num_state**2, num_state**2)  # flatten to 2-site operator

    for site in range(num_site):  # nearest-neighbor only
        ops = [np.eye(num_state)] * (num_site-1)
        # replace two adjacent sites with V
        ops[site] = None  # placeholder
        ops[site + 1] = None

        # Build kronecker with V inserted
        kron_list = []
        for idx, op in enumerate(ops):
            if idx == site:
                kron_list.append(V_matrix)
            elif op is not None:
                kron_list.append(op)
        # Apply Kronecker product
        V_Final += kron_all(kron_list)

    return V_Final

In [ ]:
def loop_site_potential(V: np.ndarray, num_sites: int, num_state: int) -> np.ndarray:

    d = num_state
    D = d ** num_sites
    H_total = np.zeros((D, D))

    for i in range(num_sites):
        j = (i + 1) % num_sites  # periodic wraparound

        # reorder so i < j for tensor layout
        i_, j_ = sorted((i, j))

        # Reshape and embed V_matrix into full space
        H = np.kron(V, np.eye(d ** (num_sites - 2)))
        H = H.reshape([d] * 2 * num_sites)      

        # H = H.transpose(perm)
        H = H.reshape(D, D)

        H_total += H

    return H_total

In [ ]:
def compute_matrix_individual_sites_V(V: np.ndarray, target_site: int, num_site: int, num_state: int)-> np.ndarray:

    matrices = [np.eye(2*num_state + 1)]*(num_site -1)

    matrices[target_site] = V

    V_sum = reduce(np.kron, matrices)

    return V_sum


def compute_nearest_neighbor_potential(V: np.ndarray, num_site: int, num_state: int)-> np.ndarray:

    V_final = np.zeros(((2*num_state + 1)**num_site, (2*num_state + 1)**num_site))

    V = V.reshape((2*num_state + 1)**2, (2*num_state + 1)**2)

    for target_site in range(num_site -1):

        V_int = compute_matrix_individual_sites_V(V, target_site, num_site, num_state)

        # hami -= V_int

        # print("Potential hami:",target_site)
        # print(hami)

        V_final += V_int

    return V_final

In [ ]:
num_sites = 3
num_states = 1

V = np.random.rand( num_states*2 +1, num_states*2 +1,  num_states*2 +1,  num_states*2 +1)

H_pot = compute_nearest_neighbor_potential(V_in_p, num_sites, num_states)
H_pot

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0

In [ ]:
def embed_two_site_operator(V, site1, site2, num_sites, d):

    site1, site2 = sorted((site1, site2)) 

    print("Target site 1:")
    print(site1)
    print("Target site 2:")
    print(site2)

    # Identity blocks
    I = np.eye(d)
    if site1 > 0:
        left = reduce(np.kron, [I] * site1)  
    else:
        left = 1

    if site2 < num_sites - 1:
        right = reduce(np.kron, [I] * (num_sites - site2 - 1)) 
    else:
        right = 1

    if site2 - site1 > 1:
        middle = np.eye(d ** (site2 - site1 - 1))
    else:
        middle = 1
    print("Left:")
    print(left)
    print("Midle:")
    print(middle)
    print("Right:")
    print(right)

    return np.kron(np.kron(left, np.kron(middle, V)), right)



def compute_nearest_neighbor_potential(V: np.ndarray, num_site: int, num_state: int)-> np.ndarray:

    V_final = np.zeros(((2*num_state + 1)**num_site, (2*num_state + 1)**num_site))

    V = V.reshape((2*num_state + 1)**2, (2*num_state + 1)**2)

    for target_site in range(num_site):

        target_site_1 = target_site
        if target_site_1 == num_site - 1:
            target_site_2 = 0
        else:
            target_site_2 = target_site + 1
        
        # print("Target site 1:")
        # print(target_site_1)
        # print("Target site 2:")
        # print(target_site_2)

        # print(embed_two_site_operator(V, target_site_1, target_site_2, num_site, 2*num_state + 1))

        V_final += embed_two_site_operator(V, target_site_1, target_site_2, num_site, 2*num_state + 1)

    return V_final

## Applying the delta function operation.(Work Kinetic with fors)

In [ ]:
def build_site_match_matrix(states: int, sites: int, site_index: int) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim), dtype=int)

    matrix_n = np.array([[0, 0, 0],
    [0, 1, 0],
    [0, 0, 1]])

    # Generate all basis state combinations as digit vectors
    basis_states = []
    for n in range(dim):
        digits = []
        x = n
        for _ in range(sites):
            digits.append(x % states)
            x //= states
        digits.reverse()
        basis_states.append(digits)

    # Fill matrix: compare state at site_index
    for seg_i in range(sites + 1):
        for seg_j in range(sites + 1):
            for i in range(states):
                for j in range(states):
                    # print(i + states*dim, "|", j + states*dim)
                    
                    if basis_states[i + states*seg_i][site_index] == basis_states[j + states*seg_j][site_index] & basis_states[i + states*seg_i][site_index + 1] == basis_states[j + states*seg_j][site_index + 1]:

                        print(basis_states[i + states*seg_i - 1][site_index], "|", basis_states[j + states*seg_j][site_index - 1])

                        matrix[i + states*seg_i, j + states*seg_j] = matrix_n[basis_states[i + states*seg_i - 1][site_index],basis_states[j + states*seg_j][site_index - 1]]

    return matrix

In [ ]:
def build_site_match_matrix(states: int, sites: int, site_index: int) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim), dtype=int)

    # Define the logic matrix
    matrix_n = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])

    # Helper: get digit at a given site for a basis index
    def get_digit(index: int, site: int) -> int:
        return (index // (states ** (sites - site - 1))) % states

    # Determine which two sites to use for comparison (not equal to site_index)
    site_indices = [i for i in range(sites) if i != site_index]
    assert len(site_indices) == 2, "Only supports 3-site systems."

    a_site0, a_site1 = site_indices

    for i in range(dim):
        for j in range(dim):
            # Extract the digits we care about
            a0 = get_digit(i, a_site0)
            a1 = get_digit(i, a_site1)
            b0 = get_digit(j, a_site0)
            b1 = get_digit(j, a_site1)

            # Apply your condition and logic matrix
            if a0 == b0 and a1 == b1:
                matrix[i, j] = matrix_n[a0, b0]  # you could also use matrix_n[a0, b0] or any logic

    return matrix

In [ ]:
def config_to_index(config, states):
    index = 0
    sites = len(config)
    for i in range(sites):
        index += config[i] * (states ** (sites - i - 1))
    return index

def get_matrix_index(unchanged, changed, states):
    row = config_to_index(unchanged, states)
    col = config_to_index(changed, states)
    return row, col

In [ ]:
def config_to_index(config, states):
    index = 0
    sites = len(config)
    for i in range(sites):
        index += config[i] * (states ** (sites - i - 1))
    return index

def index_to_config(index, states, sites):
    config = []
    for _ in range(sites):
        config.append(index % states)
        index //= states
    return config[::-1]

def build_matching_matrix(states: int, sites: int, compare_site: int) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim), dtype=int)

    for i in range(dim):
        unchanged = index_to_config(i, states, sites)

        for s in range(states):
            changed = unchanged.copy()
            changed[compare_site] = s  # vary only the compared site

            row = config_to_index(unchanged, states)
            col = config_to_index(changed, states)
            matrix[row, col] = 1

    return matrix

In [ ]:
states = 3
sites = 3
compare_site = 0

M = build_matching_matrix(states, sites, compare_site)
M

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [ ]:
def config_to_index(config, states):
    index = 0
    sites = len(config)
    for i in range(sites):
        index += config[i] * (states ** (sites - i - 1))
    return index

def index_to_config(index, states, sites):
    config = []
    for _ in range(sites):
        config.append(index % states)
        index //= states
    return config[::-1]

def build_weighted_site_matrix(states: int, sites: int, compare_site: int, matrix_n: np.ndarray) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim))

    for i in range(dim):
        unchanged = index_to_config(i, states, sites)

        for j in range(dim):
            changed = index_to_config(j, states, sites)

            # compare all sites except compare_site
            if all(unchanged[k] == changed[k] for k in range(sites) if k != compare_site):
                a = unchanged[compare_site]
                b = changed[compare_site]
                matrix[i, j] = matrix_n[a, b]

    return matrix

In [ ]:
states = 3
sites = 3
compare_site = 3

M = np.zeros((states**sites, states**sites))

# Example state-by-state logic matrix (you can use any weights here)
matrix_n = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
for i in range(compare_site):
    M += build_weighted_site_matrix(states, sites, i, matrix_n)
print(M)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
def build_weighted_site_matrix(states: int, sites: int, compare_site: int, matrix_n: np.ndarray) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim))

    for i in range(dim):
        for j in range(dim):
            match = True

            # Compare all digits except the compare_site
            for s in range(sites):
                if s == compare_site:
                    continue
                # extract digit at site s for i and j
                i_digit = (i // states**(sites - s - 1)) % states
                j_digit = (j // states**(sites - s - 1)) % states

                if i_digit != j_digit:
                    match = False
                    break

            if match:
                a = (i // states**(sites - compare_site - 1)) % states
                b = (j // states**(sites - compare_site - 1)) % states
                matrix[i, j] = matrix_n[a, b]

    return matrix

In [ ]:
states = 3
sites = 3
compare_site = 3

M = np.zeros((states**sites, states**sites))

# Example state-by-state logic matrix (you can use any weights here)
matrix_n = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
for i in range(compare_site):
    M += build_weighted_site_matrix(states, sites, i, matrix_n)
print(M)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
import numpy as np
from itertools import product

def config_to_index(config, states):
    index = 0
    sites = len(config)
    for i in range(sites):
        index += config[i] * (states ** (sites - i - 1))
    return index

def get_matrix_index(unchanged, changed, states):
    row = config_to_index(unchanged, states)
    col = config_to_index(changed, states)
    return row, col

def build_weighted_site_matrix_sparse(states: int, sites: int, compare_site: int, matrix_n: np.ndarray) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim))

    # Iterate over all possible configurations (unchanged)
    for config in product(range(states), repeat=sites):
        unchanged = list(config)
        print(unchanged)
        for s in range(states):
            changed = unchanged.copy()
            changed[compare_site] = s

            row, col = get_matrix_index(unchanged, changed, states)
            a = unchanged[compare_site]
            b = changed[compare_site]
            matrix[row, col] = matrix_n[a, b]

    return matrix

In [ ]:
def config_to_index(config, states):
    index = 0
    sites = len(config)
    for i in range(sites):
        index += config[i] * (states ** (sites - i - 1))
    return index

def get_matrix_index(unchanged, changed, states):
    row = config_to_index(unchanged, states)
    col = config_to_index(changed, states)
    return row, col

def build_weighted_site_matrix_sparse(states: int, sites: int, compare_site: int, matrix_n: np.ndarray) -> np.ndarray:
    dim = states ** sites
    matrix = np.zeros((dim, dim))

    # Iterate over all possible configurations (unchanged)
    for config in product(range(states), repeat=sites):
        unchanged = list(config)
        a = unchanged[compare_site]

        for b in range(states):
            val = matrix_n[a, b]
            if val == 0:
                continue  # Skip if matrix_n entry is zero

            changed = unchanged.copy()
            changed[compare_site] = b

            row, col = get_matrix_index(unchanged, changed, states)
            matrix[row, col] = val

    return matrix

## Applying the delta function operation.(Final Kitentik with fors)

In [5]:
def H_kinetic(states: int, sites: int, h_pp: np.ndarray) -> np.ndarray:

    K = np.zeros((states**sites, states**sites))

    for x in range(sites):
        n_lambda = states**(x)
        n_mu = states**(sites - x - 1)

        for p in range(states):
            for p_prime in range(states):
                val = h_pp[p, p_prime]
                if val == 0:
                    continue  # skip writing 0s
                for Lambda in range(int(n_lambda)):
                    for mu in range(int(n_mu)):
                        i = mu + p * n_mu + Lambda * states * n_mu
                        j = mu + p_prime * n_mu + Lambda * states * n_mu
                        K[i, j] += val
    return K

In [ ]:
states = 3
sites = 3
compare_site = 3

# Example state-by-state logic matrix (you can use any weights here)
K = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
N = H_kin(states, sites, K)
print(N)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Applying the delta function operation.(Final Potential with fors)

In [78]:
def H_potential(states: int, sites: int, h_pp_qq: np.ndarray) -> np.ndarray:

    K = np.zeros((states**sites, states**sites))

    for x in range(sites):
        y = (x+1) % sites
        n_lambda = states**(x % (sites-1))
        n_mu = states**((sites - y - 1) % (sites - 1))
        n_nu = states**(np.abs(y - x) - 1)

        for q in range(states):
            for q_prime in range(states):
                for p in range(states):
                    for p_prime in range(states):

                        val = h_pp_qq[p,q, p_prime, q_prime]
                        if val == 0:
                            continue  # skip writing 0s

                        for Lambda in range(int(n_lambda)):
                            for mu in range(int(n_mu)):
                                for nu in range(int(n_nu)):
                                    
                                    i = mu + q*n_mu + nu*states*n_mu + p*n_nu*n_mu*states + Lambda*n_nu*n_mu*states**2
                                    j = mu + q_prime*n_mu + nu*states*n_mu + p_prime*n_nu*n_mu*states + Lambda*n_nu*n_mu*states**2
                                    K[i, j] += val
    return K

In [69]:
x = 5
sites = 6
states = 5


for x in range(sites):

    y_1 = (x+1) % sites
    n_lambda_1 = x
    n_mu_1 = sites - y_1 - 1
    n_nu_1 = y_1 - x - 1

    if x == sites -1:
        n_lambda_1 = 0
        n_mu_1 = 0
        n_nu_1 = sites-2

    y = ((x+1) % sites)
    n_lambda = x % (sites-1)
    n_mu = (sites - y - 1) % (sites - 1)
    n_nu = np.abs(y - x) - 1

    print("x: ", x)
    print("y: ", y, "y_1: ", y_1, "Comp_y: ", y == y_1)
    print("Lambda: ", n_lambda, "Lambda_1: ", n_lambda_1, "Comp_Lambda: ", n_lambda == n_lambda_1)
    print("Mu: ", n_mu, "Mu_1: ", n_mu_1, "Comp_Mu: ", n_mu == n_mu_1)
    print("Nu: ", n_nu, "Nu_1: ", n_nu_1, "Comp_Nu: ", n_nu == n_nu_1)

x:  0
y:  1 y_1:  1 Comp_y:  True
Lambda:  0 Lambda_1:  0 Comp_Lambda:  True
Mu:  4 Mu_1:  4 Comp_Mu:  True
Nu:  0 Nu_1:  0 Comp_Nu:  True
x:  1
y:  2 y_1:  2 Comp_y:  True
Lambda:  1 Lambda_1:  1 Comp_Lambda:  True
Mu:  3 Mu_1:  3 Comp_Mu:  True
Nu:  0 Nu_1:  0 Comp_Nu:  True
x:  2
y:  3 y_1:  3 Comp_y:  True
Lambda:  2 Lambda_1:  2 Comp_Lambda:  True
Mu:  2 Mu_1:  2 Comp_Mu:  True
Nu:  0 Nu_1:  0 Comp_Nu:  True
x:  3
y:  4 y_1:  4 Comp_y:  True
Lambda:  3 Lambda_1:  3 Comp_Lambda:  True
Mu:  1 Mu_1:  1 Comp_Mu:  True
Nu:  0 Nu_1:  0 Comp_Nu:  True
x:  4
y:  5 y_1:  5 Comp_y:  True
Lambda:  4 Lambda_1:  4 Comp_Lambda:  True
Mu:  0 Mu_1:  0 Comp_Mu:  True
Nu:  0 Nu_1:  0 Comp_Nu:  True
x:  5
y:  0 y_1:  0 Comp_y:  True
Lambda:  0 Lambda_1:  0 Comp_Lambda:  True
Mu:  0 Mu_1:  0 Comp_Mu:  True
Nu:  4 Nu_1:  4 Comp_Nu:  True


# Converting from m to p.

## Free one body syste m to p. (Work in progress)

In [ ]:

# Original matrix A
A = np.array([
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10]
])
print(A)
# Index map: new_index[i] = index of row i in the new basis
index_map = [0, 3, 1, 2]  # Example: remap row/col 1 → 3, 2 → 1, 3 → 2

# Permute rows and columns accordingly
A_new = A[np.ix_(index_map, index_map)]
A_new = A_new.T[np.ix_(index_map, index_map)]

print(np.ix_(index_map, index_map))

print(A_new)

P = np.eye(len(index_map))[index_map]
A_new2 = P.T @ A @ P

print(A_new2)

[[ 1  2  3  4]
 [ 2  5  6  7]
 [ 3  6  8  9]
 [ 4  7  9 10]]
(array([[0],
       [3],
       [1],
       [2]]), array([[0, 3, 1, 2]]))
[[ 1  3  4  2]
 [ 3  8  9  6]
 [ 4  9 10  7]
 [ 2  6  7  5]]
[[ 1.  3.  4.  2.]
 [ 3.  8.  9.  6.]
 [ 4.  9. 10.  7.]
 [ 2.  6.  7.  5.]]


In [ ]:
def free_one_body(max_m: int)-> np.array:

    p_max = 2*max_m + 1
    p_vect = np.zeros(p_max)

    m = 0

    for i in range(1, max_m+1):

        p_vect[i + m] = i**2
        p_vect[i + 1 + m] = (-i)**2
        m+=1
    
    print(p_vect)

    return  np.diag(p_vect)

K_new = free_one_body(5)

K_new

[ 0.  1.  1.  4.  4.  9.  9. 16. 16. 25. 25.]


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25.]])

In [ ]:
df = pd.read_csv(
    "/Users/gilfrim/Desktop/QuantumChemistryCoop/Main-CC-files/matrix_elements_K.csv",
    skiprows=1, header=0
)

# Clean column names
df.columns = [col.strip() for col in df.columns]

# Convert columns safely
df["m1"] = df["m1"].astype(int)
df["m2"] = df["m2"].astype(int)
df["<m1|K|m2>"] = df["<m1|K|m2>"].astype(float)

# Matrix size and init
size = max(df["m1"].max(), df["m2"].max()) + 1
K = np.zeros((size, size))

m = (size - 1) / 2

# Fill matrix
for _, row in df.iterrows():

    i = int(row["m1"])
    j = int(row["m2"])
    val = float(row["<m1|K|m2>"])
    K[i, j] = val
    K[j, i] = val

    
# Show matrix
print(pd.DataFrame(K))

      0     1    2    3    4    5    6    7    8     9     10
0   25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
1    0.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
2    0.0   0.0  9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
3    0.0   0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
4    0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0
5    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
6    0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   0.0   0.0
7    0.0   0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0   0.0   0.0
8    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0   0.0
9    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  16.0   0.0
10   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  25.0


In [ ]:
# p_vect = np.zeros(11)
p_vect = np.array([0,  -1, 1,  -2, 2,  -3, 3,  -4, 4,  -5, 5])
m=0
vec1 = np.zeros(11)

for i in range(11):
    vec1[i] = int(func.m_to_p(p_vect[i]))

print(p_vect)
print(vec1)

[ 0 -1  1 -2  2 -3  3 -4  4 -5  5]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


## Free one body system m to p. (Final Function)

In [ ]:
def create_inverse_index_map(total_num_states: int) -> np.ndarray:
    # map m → p
    m_vals = np.arange(-total_num_states, total_num_states + 1)
    p_vals = np.vectorize(func.m_to_p)(m_vals)

    # Create inverse: map p → position in m-sorted order
    inverse_index_map = np.zeros_like(p_vals)
    for i, p in enumerate(p_vals):
        inverse_index_map[p] = i

    return inverse_index_map

def basis_m_to_p_matrix_conversion_one_body(matrix: np.ndarray)->np.ndarray:

    dim = matrix.shape[0]
    index_map = create_inverse_index_map((dim-1)//2)

    matrix = matrix[np.ix_(index_map, index_map)]

    return matrix

print(pd.DataFrame(basis_m_to_p_matrix_conversion_one_body(K)))

     0    1    2    3    4    5    6     7     8     9     10
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
1   0.0  1.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
2   0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
3   0.0  0.0  0.0  4.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
4   0.0  0.0  0.0  0.0  4.0  0.0  0.0   0.0   0.0   0.0   0.0
5   0.0  0.0  0.0  0.0  0.0  9.0  0.0   0.0   0.0   0.0   0.0
6   0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0   0.0   0.0   0.0
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  16.0   0.0   0.0   0.0
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  16.0   0.0   0.0
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  25.0   0.0
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0  25.0


In [ ]:
df = pd.read_csv(
    "/Users/gilfrim/Desktop/QuantumChemistryCoop/Main-CC-files/matrix_elements_K.csv",
    skiprows=1, header=0
)

# Clean column names
df.columns = [col.strip() for col in df.columns]

# Convert columns safely
df["m1"] = df["m1"].astype(int)
df["m2"] = df["m2"].astype(int)
df["<m1|K|m2>"] = df["<m1|K|m2>"].astype(float)

# Matrix size and init
size = max(df["m1"].max(), df["m2"].max()) + 1
K = np.zeros((size, size))

m = (size - 1) / 2

# Fill matrix
for _, row in df.iterrows():

    i = int(row["m1"])
    j = int(row["m2"])
    val = float(row["<m1|K|m2>"])
    K[i, j] = val
    K[j, i] = val

    
# Show matrix
print(pd.DataFrame(K))

print(pd.DataFrame(basis_m_to_p_matrix_conversion_one_body(K)))

      0     1    2    3    4    5    6    7    8     9     10
0   25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
1    0.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
2    0.0   0.0  9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
3    0.0   0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
4    0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0
5    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
6    0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   0.0   0.0
7    0.0   0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0   0.0   0.0
8    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0   0.0
9    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  16.0   0.0
10   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  25.0
     0    1    2    3    4    5    6     7     8     9     10
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
1   0.0  1.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
2   0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0
3   0.0 

## Coplaner two body system. (Work in progress)

### Learning einsum

1. Inner product of vectors

'i,i->': sum over index i (vector inner product)

In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

np.dot(a, b)         # → 32
np.einsum('i,i->', a, b)  # → 32

32

2. Matrix multiplication

'ik,kj->ij': sum over index k (standard matrix multiplication)

In [ ]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[2, 0], [1, 2]])

np.matmul(A, B)
np.einsum('ik,kj->ij', A, B)

array([[ 4,  4],
       [10,  8]])

3. Trace of a matrix

'ii->': sum diagonal elements (same index repeated = sum)

In [ ]:
a = np.array([1, 2])
b = np.array([3, 4, 5])

np.einsum('i,j->ij', a, b)
# Result: 2x3 matrix of a[i] * b[j]

array([[ 3,  4,  5],
       [ 6,  8, 10]])

4. Tensor contraction

Contracts the last index of T with w (like applying weights)

In [ ]:
T = np.random.rand(2, 3, 4)
w = np.random.rand(4)

np.einsum('ijk,k->ij', T, w)

array([[0.93882073, 0.96995704, 1.08514116],
       [0.734377  , 0.76955526, 0.76790332]])

5. Outer product

In [ ]:
a = np.array([1, 2])
b = np.array([3, 4, 5])

np.einsum('i,j->ij', a, b)
# Result: 2x3 matrix of a[i] * b[j]

array([[ 3,  4,  5],
       [ 6,  8, 10]])

###

In [ ]:

T = np.arange(3**2).reshape(3, 3)
print("Original Tensor (flattened):")
print(T.flatten())

for i in range(1, 4):
    for j in range(1, 4):
        T[i-1,j-1] = 100*i+10*j

index_map = [1, 0, 2]  # Permute axis 0
index_maps = []
for i in range(2):
    index_maps.append(index_map)

print(index_maps)
# Unpack index_maps into np.ix_
T_permuted = T[np.ix_(*index_maps)]

P = np.eye(len(index_map))[index_map]
T_einsum = np.einsum('ai,bj,ij->ab', P, P, T)

print(P.shape)
print(T.shape)

T_change_of_bais = P.T @ T @ P

print("Original slice T[0, 1, :, :]:")
print(T)

print("\nPermuted slice T_permuted[0, 1, :, :]:")
print(T_permuted)
print(T_einsum)
print(T_change_of_bais)


Original Tensor (flattened):
[0 1 2 3 4 5 6 7 8]
[[1, 0, 2], [1, 0, 2]]
(3, 3)
(3, 3)
Original slice T[0, 1, :, :]:
[[110 120 130]
 [210 220 230]
 [310 320 330]]

Permuted slice T_permuted[0, 1, :, :]:
[[220 210 230]
 [120 110 130]
 [320 310 330]]
[[220. 210. 230.]
 [120. 110. 130.]
 [320. 310. 330.]]
[[220. 210. 230.]
 [120. 110. 130.]
 [320. 310. 330.]]


In [ ]:

T = np.arange(3**3).reshape(3, 3, 3)
print("Original Tensor (flattened):")
print(T.flatten())

for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 4):
            T[i-1,j-1,k-1] = 100*i+10*j+k

index_map = [1, 0, 2]  # Permute axis 0
index_maps = []
for i in range(3):
    index_maps.append(index_map)

print(index_maps)
# Unpack index_maps into np.ix_
T_permuted = T[np.ix_(*index_maps)]

print("Original slice T[0, 1, :, :]:")
print(T[0, :, :])

print("\nPermuted slice T_permuted[0, 1, :, :]:")
print(T_permuted[0, :, :])

print(index_maps)

Original Tensor (flattened):
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]
[[1, 0, 2], [1, 0, 2], [1, 0, 2]]
Original slice T[0, 1, :, :]:
[[111 112 113]
 [121 122 123]
 [131 132 133]]

Permuted slice T_permuted[0, 1, :, :]:
[[222 221 223]
 [212 211 213]
 [232 231 233]]
[[1, 0, 2], [1, 0, 2], [1, 0, 2]]


In [ ]:
T

array([[[111, 112, 113],
        [121, 122, 123],
        [131, 132, 133]],

       [[211, 212, 213],
        [221, 222, 223],
        [231, 232, 233]],

       [[311, 312, 313],
        [321, 322, 323],
        [331, 332, 333]]])

## Coplaner two body system. (Final Function)

In [10]:
def create_inverse_index_map(total_num_states: int) -> np.ndarray:
    # map m → p
    m_vals = np.arange(-total_num_states, total_num_states + 1)
    p_vals = np.vectorize(func.m_to_p)(m_vals)

    # Create inverse: map p → position in m-sorted order
    inverse_index_map = np.zeros_like(p_vals)
    for i, p in enumerate(p_vals):
        inverse_index_map[p] = i

    return inverse_index_map

def basis_m_to_p_matrix_conversion(V: np.ndarray)->np.ndarray:

    dim = V.ndim
    index_map = create_inverse_index_map((V.shape[0]-1)//2)

    index_maps = []
    for i in range(dim):
        index_maps.append(index_map)

    V = V[np.ix_(*index_maps)]

    return V

# Test Done Code.

In [12]:
def write_matrix_elements(m_max, fpath):
    d = 2 * m_max + 1

    # Generate Kinetic Energy Matrix
    K = np.zeros((d, d))
    for i in range(d):
        for j in range(i, d):
            K[i, j] = hg.free_one_body(i, j, m_max)

    # Save K as a NumPy binary file
    np.save(os.path.join(fpath, "K_matrix.npy"), K)

    # Generate Potential Energy Matrix (flattened 2-body operator)
    V = np.zeros((d**2, d**2))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for l in range(d):
                    if k * d + l >= i * d + j:  # upper triangular only
                        V[i*d + j, k*d + l] = hg.interaction_two_body_coplanar(i, j, k, l)

    # Save V as a NumPy binary file
    np.save(os.path.join(fpath, "V_matrix.npy"), V)

    return K, V  # optionally return them for immediate use

In [ ]:
states_new = 11
sites_new = 3

write_matrix_elements((states_new-1) // 2, r"/Users/gilfrim/Desktop/Computational_Analisis_of_Many_Rotor_Systems/Main-CC-files")

K_path = os.path.join(r"/Users/gilfrim/Desktop/Computational_Analisis_of_Many_Rotor_Systems/Main-CC-files", "K_matrix.npy")
V_path = os.path.join(r"/Users/gilfrim/Desktop/Computational_Analisis_of_Many_Rotor_Systems/Main-CC-files", "V_matrix.npy")

K_from_csv = np.load("K_matrix.npy")
V_from_csv = np.load("V_matrix.npy")

V_from_csv = V_from_csv + V_from_csv.T - np.diag(np.diag(V_from_csv))
V_tensor = V_from_csv.reshape(states_new, states_new, states_new, states_new)  # Adjust if needed

K_in_p = basis_m_to_p_matrix_conversion(K_from_csv)
V_in_p = basis_m_to_p_matrix_conversion(V_tensor)

K_final = H_kinetic(states_new, sites_new, K_in_p)
V_final = H_potential(states_new, sites_new, V_in_p)
H_full = HB.hamiltonian(sites_new, states_new)

try:
    os.remove(K_path)
    os.remove(V_path)
    print("Temporary files deleted.")
except FileNotFoundError as e:
    print(f"File not found during deletion: {e.filename}")

comparison = np.array_equal(H_full, K_final + V_final)

print("Result:", comparison)

The time of execution of above program is : 0.17661904194392264 s
Temporary files deleted.
Result: True


In [ ]:
for sites_new in range(2, 6):
    for states_new in range(3, 6, 2):
        print(f"\nTesting (states={states_new}, sites={sites_new})")

        write_matrix_elements((states_new-1) // 2, r"/Users/gilfrim/Desktop/Computational_Analisis_of_Many_Rotor_Systems/Main-CC-files")

        # Load files
        K_path = os.path.join(r"/Users/gilfrim/Desktop/Computational_Analisis_of_Many_Rotor_Systems/Main-CC-files", "K_matrix.npy")
        V_path = os.path.join(r"/Users/gilfrim/Desktop/Computational_Analisis_of_Many_Rotor_Systems/Main-CC-files", "V_matrix.npy")

        K_from_csv = np.load("K_matrix.npy")
        V_from_csv = np.load("V_matrix.npy")

        V_from_csv = V_from_csv + V_from_csv.T - np.diag(np.diag(V_from_csv))
        V_tensor = V_from_csv.reshape(states_new, states_new, states_new, states_new)  # Adjust if needed

        K_in_p = basis_m_to_p_matrix_conversion(K_from_csv)
        V_in_p = basis_m_to_p_matrix_conversion(V_tensor)

        K_final = H_kinetic(states_new, sites_new, K_in_p)
        V_final = H_potential(states_new, sites_new, V_in_p)
        H_full = HB.hamiltonian(sites_new, states_new, timer = False)

        comparison = np.array_equal(H_full, K_final + V_final)

        print("Result K:", comparison)

        # Delete temporary files
        try:
            os.remove(K_path)
            os.remove(V_path)
        except FileNotFoundError as e:
            print(f"File not found during deletion: {e.filename}")


Testing (states=3, sites=2)
Result K: True

Testing (states=5, sites=2)
Result K: True

Testing (states=3, sites=3)
Result K: True

Testing (states=5, sites=3)
Result K: True

Testing (states=3, sites=4)
Result K: True

Testing (states=5, sites=4)
Result K: True

Testing (states=3, sites=5)
Result K: True

Testing (states=5, sites=5)
Result K: True
